Name: Khushank Mehta
Enigma Securities Internship Exercise

In [1]:
import websocket
import json
import time
import csv

# Defining the WebSocket endpoint
socket = "wss://api-pub.bitfinex.com/ws/2"
symbols = {"BTCUSD": "tBTCUSD", "ETHUSD": "tETHUSD", "BTCF0USTF0": "BTCF0:USTF0", "ETHF0USTF0": "ETHF0:USTF0"}
symbol = input("Enter Symbol (BTCUSD, ETHUSD, BTCF0USTF0, ETHF0USTF0): ")

csv_file_path = f'{symbol}.csv' # Creating our csv file with specific ticker name

# Initialize counters to calculate number of trades, trade volume, number of quotes
num_trades = 0
total_volume = 0
num_quotes = 0

# subscribing the symbol to get ticker data (Picked this code from api documentation of bitfinex)
subscribe_message = json.dumps({
    "event": "subscribe",
    "channel": 'ticker',
    "symbol": symbols.get(symbol.upper())
})


# Creating dictionaries for bid and ask prices in order to store price values for each level quantity
bid_prices = {1: [], 10: [], 25: []}
ask_prices = {1: [], 10: [], 25: []}

# processing incoming messages to get the ticker data in required json format
def on_message(ws, message):
    global num_trades, total_volume, num_quotes
    data = json.loads(message)

    if isinstance(data, list) and len(data) > 1:  #This condition checks if the incoming data is a list and has more than 1 values to ensure that data is extracted in correct format.
        ticker_data = data[1] # Here the indexing 1 signifies that we extract the second list which contains bid prices, ask prices, volume details for the specific ticker
        if isinstance(ticker_data, list) and len(ticker_data) == 10:  #This code checks whether the data has all necessary informations without missing any values
            num_trades += 1
            total_volume += float(ticker_data[7])  # volume is at index 7 
            num_quotes += 1  # Assuming each tick is a quote

            bid_price = float(ticker_data[0]) # The indexing 0 extracts the bid price
            ask_price = float(ticker_data[2]) # The indexing 2 extracts the ask price
            
            # Calculating bid and ask prices for different level quantities
            for level_quantity in [1, 10, 25]:
                bid_prices[level_quantity].append(bid_price - level_quantity * 0.1)
                ask_prices[level_quantity].append(ask_price + level_quantity * 0.1)

            # Calculating the average bid and ask prices for each level quantity
            avg_bid = {level_quantity: sum(prices) / len(prices) for level_quantity, prices in bid_prices.items()}
            avg_ask = {level_quantity: sum(prices) / len(prices) for level_quantity, prices in ask_prices.items()}

            # Calculating the average spread for each level quantity
            avg_spread = {level_quantity: avg_ask[level_quantity] - avg_bid[level_quantity] for level_quantity in bid_prices.keys()}

            # Calculating the average volume
            avg_volume = total_volume / num_trades

            #  # Writing if-else statement to specify whether the ticker is spot or perpetual and also mention the level Quantity
            if symbol == "BTCUSD" or symbol == "ETHUSD":
                for level_quantity in [1, 10, 25]:
                    response = {
                        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
                        "instrument": symbol,
                        "type": "Spot",
                        "level_quantity": level_quantity,
                        "avg_bid": avg_bid[level_quantity],
                        "avg_ask": avg_ask[level_quantity],
                        "num_trades": num_trades,
                        "total_volume": total_volume,
                        "num_quotes": num_quotes,
                        "avg_volume": avg_volume,
                        "avg_spread [1, 10, 25]": avg_spread[level_quantity]
                    }
                    with open(csv_file_path, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        # # specifying the columns in our csv
                        writer.writerow(response.keys())
                        # printing values for specific columns in our csv
                        writer.writerow(response.values())
                    print(json.dumps(response, indent=4))
            else:
                for level_quantity in [1, 10, 25]:
                    response = {
                        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
                        "instrument": symbol,
                        "type": "Perpetual",
                        "level_quantity": level_quantity,
                        "avg_bid": avg_bid[level_quantity],
                        "avg_ask": avg_ask[level_quantity],
                        "num_trades": num_trades,
                        "total_volume": total_volume,
                        "num_quotes": num_quotes,
                        "avg_volume": avg_volume,
                        "avg_spread [1, 10, 25]": avg_spread[level_quantity]
                    }
                    with open(csv_file_path, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        # specifying the column in our csv
                        writer.writerow(response.keys())
                        # printing values for specific columns in our csv
                        writer.writerow(response.values())
                    print(json.dumps(response, indent=4))

            

# WebSocket connection close
def on_close(ws):
    print("closed")


# WebSocket connection open
def on_open(ws):
    print("open")
    ws.send(subscribe_message)

# Creating a WebSocket connection to our api link
ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close, on_open=on_open)
ws.run_forever()



Enter Symbol (BTCUSD, ETHUSD, BTCF0USTF0, ETHF0USTF0): BTCUSD
open
{
    "timestamp": "2024-04-01 17:21:08",
    "instrument": "BTCUSD",
    "type": "Spot",
    "level_quantity": 1,
    "avg_bid": 69810.9,
    "avg_ask": 69812.1,
    "num_trades": 1,
    "total_volume": 2647.69819543,
    "num_quotes": 1,
    "avg_volume": 2647.69819543,
    "avg_spread [1, 10, 25]": 1.2000000000116415
}
{
    "timestamp": "2024-04-01 17:21:08",
    "instrument": "BTCUSD",
    "type": "Spot",
    "level_quantity": 10,
    "avg_bid": 69810.0,
    "avg_ask": 69813.0,
    "num_trades": 1,
    "total_volume": 2647.69819543,
    "num_quotes": 1,
    "avg_volume": 2647.69819543,
    "avg_spread [1, 10, 25]": 3.0
}
{
    "timestamp": "2024-04-01 17:21:08",
    "instrument": "BTCUSD",
    "type": "Spot",
    "level_quantity": 25,
    "avg_bid": 69808.5,
    "avg_ask": 69814.5,
    "num_trades": 1,
    "total_volume": 2647.69819543,
    "num_quotes": 1,
    "avg_volume": 2647.69819543,
    "avg_spread [1, 10, 2

False